In [1]:
import pandas as pd
from glob import glob
import json
import os
import re

In [5]:
def get_filelst(SAVE_PATH):
    return list(set([i.replace("_0_shot.json","") for i in os.listdir(SAVE_PATH)]))

# results/korean_origin_bench/20b/00_shot.json
def _get_metric_name(v):
    metrics = ['f1', 'macro_f1', 'acc_norm', 'acc']
    for m in metrics:
        if v.get(m):
            return {
                'metric': m,
                'value': v[m],
            }

def get_df_klue(SAVE_PATH, model_name=''):
    
    data = []
    for i in ['0', '5', '10', '50']:
        shot = f'{SAVE_PATH}/{model_name}_{i}_shot.json'
        try:

            data.append(
                {   
                    # 'kobest_boolq (macro_f1)': 0.35461338879835325
                    f"{k} ({_get_metric_name(v)['metric']})": _get_metric_name(v)['value']
                    for k, v in json.load(open(shot))['results'].items()
                }
            )
        except FileNotFoundError:
            pass
    df = pd.DataFrame(data, index=[model_name]).T
    return df

In [6]:
SAVE_PATH = '../data/result'
filelst = get_filelst(SAVE_PATH)
print("file 개수 : ",len(filelst))
temp_df = []
for file in filelst:
    if re.match("^\(ours\)",file):
        continue
    df = get_df_klue(SAVE_PATH,model_name=file)
    temp_df.append(df)
print("일부 제거 후 file 개수 : ",len(temp_df))
all_df = pd.concat(temp_df,axis=1).T.reset_index()
all_df.rename(columns={"index":"name"},inplace=True)
all_df.to_csv("data.csv",index=False)

file 개수 :  21
일부 제거 후 file 개수 :  18


In [7]:
import pandas as pd
import pygwalker as pyg
df = pd.read_csv('./data.csv')
walker = pyg.walk(df,hideDataSourceConfig=True, vegaTheme='vega')

Box(children=(HTML(value='<div id="ifr-pyg-000615020c686689k2ZzqeITpsVbt3vc" style="height: auto">\n    <head>…